In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA
import os
from jilm.model import JILMLangModel
from jilm.document_loader import DocumentLoader
from jilm.settings import CHROMA_SETTINGS, PERSIST_DIRECTORY

In [ ]:
embeddings_model_name = os.environ.get('EMBEDDINGS_MODEL_NAME')
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)


In [ ]:
doc = DocumentLoader.load_single_document("../README.md")


In [ ]:
# Create and store locally vectorstore
db = Chroma.from_documents([doc], embeddings, persist_directory=PERSIST_DIRECTORY, client_settings=CHROMA_SETTINGS)
db.persist()
db = None

In [ ]:
retriever = db.as_retriever()

In [ ]:
#llm = JILMLangModel(retriever=retriever, embeddings=embeddings, max_tokens=1000, chunk_size=64, chunk_overlap=0)
llm = JILMLangModel(callbacks=[StreamingStdOutCallbackHandler()], retriever=retriever, embeddings=embeddings, max_tokens=1000, chunk_size=64, chunk_overlap=0)

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

In [ ]:
query = "What is the name of the project?"
res = qa(query)    
answer, docs = res['result'], res['source_documents']

In [ ]:
for document in docs:
    print("\n> " + document.metadata["source"] + ":")
    print(document.page_content)